In [1]:
import numpy as np
from scipy.stats import unitary_group
#fixed!!
nDim = 2
Ns = 2
Nc = 2

Nx = 4
Nt = 4
lattice_volume = Nt * Nx

In [2]:
Pauli = []
Pauli.append(np.array([[0, 1], [1, 0]]))
Pauli.append(np.array([[0,-1j], [1j, 0]]))
Pauli.append(np.array([[1, 0], [0, -1]]))
Id = np.array([[1, 0], [0, 1]])

In [3]:
def Transpose(array):
    axes = np.arange(len(array.shape))
    axes[-2:] = np.flip(axes[-2:]) 
    return np.transpose(array, axes = axes)

In [4]:
def ConjugateTranspose(array):
    axes = np.arange(len(array.shape))
    axes[-2:] = np.flip(axes[-2:]) 
    return np.conjugate(np.transpose(array, axes = axes))

In [5]:
def RandomSU2matrix():
    tmp = np.random.rand(3)
    a = np.sqrt(sum(tmp*tmp))
    tmp /= a
    out = np.array([[0.j,0.j],[0.j,0.j]])
    for i in range(3):
        out += Pauli[i] * tmp[i]
    out *= np.sin(a) * 1j
    return Id * np.cos(a) + out

Dirac Lagrangian on the lattice using Wilson fermions:

$\overline{\psi}_x D_{xy}[U] \psi_y = \overline{\psi}_x \left[ (m_q+2) \delta_{xy} -\frac{1}{2}\sum_{\mu} \left(\Gamma_{+\mu} U_\mu(x) \delta_{x+\hat\mu,y} + \Gamma_{-\mu} U^\dagger_\mu(x-\hat\mu) \delta_{x-\hat\mu,y} \right) \right] \psi_y$,

where $\Gamma_{\pm \mu} = \mathbb{1} \mp \gamma_\mu$, $\gamma_\mu$ are the Dirac $\gamma$-matrices, $U_\mu(x)$ is the gauge link at site $x$ pointing at direction $\mu$, and $\hat\mu$ is a unit vector pointing in direction $\mu$.

Note that the $\Gamma$'s are $2 \times 2$ matrices in 2 spacetime dimensions. Dirac and colour indices have been suppressed in the above formula for clarity.

Calculating D, and inverting D is numerically slow, therefore we use the approximation where we can expand the inverse acting on a vector by a sum over applying D multiple times to this same vector. For this we need to Calculate D, which is sparse, and then compute the Matrix-Vector multiplication, which can be done for CSR Sparse matrices. Other way to do this (maybe not possible in the optical hardware?) is to directly apply D to the vector, and this is also implemented

In [6]:
psi = np.random.randn(lattice_volume, 2, 2)

In [7]:
gauge_links = np.zeros((lattice_volume, nDim, 2,2),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = RandomSU2matrix()

In [8]:
def LinearToLattice(array, Nt, Nx):
    return np.reshape(array, np.roll(np.append(np.array(array[0].shape), [Nx,Nt]),2))
def LatticeToLinear(array, lattice_volume):
    return np.reshape(array, np.roll(np.append(np.array(array[0,0].shape), lattice_volume),1))

# Sparse Matrix Multiplication

In [9]:
def CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli):
    #Delta_x_y, dependent only on the onsite
    diagonal = (m+2)
    
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]
    
    gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
    gauge_links = LatticeToLinear(gauge_links, lattice_volume) * (-0.5)
    gauge_links_shifted_t = LatticeToLinear(gauge_links_shifted_t, lattice_volume) * (-0.5)
    gauge_links_shifted_x = LatticeToLinear(gauge_links_shifted_x, lattice_volume) * (-0.5)
    
    
    #In each row they are 2^d (next Neighbors) + 1 (onsite)
    row_index = np.arange(0,(2**nDim+1)*lattice_volume+1, 2**nDim+1)


    col_index = np.array([])
    values_spinors = np.array([])
    values_colors = np.array([])
    for i in range(lattice_volume):
        #Just moving in time and space to the next neighbors considering periodic boundary
        col_index = np.append(col_index,
                                 np.array([(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt
                                           ,(np.floor(i/4)*4)+((i%Nx)-1)%Nx, i,
                                           (np.floor(i/4)*4)+((i%Nx)+1)%Nx, (i%Nx) +((np.floor((i/Nt))+1)%Nt)*Nt]))
        #Choose the right values depending on direction in space-time, according to the order from above in col_index
        
        values_spinors = np.append(values_spinors, np.array([offdiagonal_spinor_t_minus, 
                                         offdiagonal_spinor_x_minus, np.sqrt(diagonal)*np.identity(Ns), 
                                                             offdiagonal_spinor_x_plus, offdiagonal_spinor_t_plus]))
        #We have to use sqrt of diagonal instead of diagonal, because we perform two matrix multiplication
        
#         values_colors = np.append(values_colors, np.array([gauge_links[(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt,0].T, 
#                                          gauge_links[(np.floor(i/4)*4)+((i%Nx)-1)%Nx,1].T, diagonal*np.identity(Nc), 
#                                                              gauge_links[i,1], 
#                                                            offdiagonal_spinor_t_plus]))
        values_colors = np.append(values_colors, np.array([ConjugateTranspose(gauge_links_shifted_t[i]), 
                                         ConjugateTranspose(gauge_links_shifted_x[i]), np.sqrt(diagonal)*np.identity(Nc), 
                                                             gauge_links[i,1], gauge_links[i,0]]))
    
        #I feel the transpose should be opposite, or idk
        
        #Maybe it's smart to add a value_diagonal, and separate all three, we could eliminate the diagonal term in col
        #and then we can choose the correct gauge link using the col_index. Or maybe we could move the gauge_link to right
        #The problem with the first approach, is that the positive u dont shift the gauge link. Therefore, maybe only
        #considere specific elements of col_index, namely the minus part and the diagonal part. 
        
        
    values_spinors = np.reshape(values_spinors, (lattice_volume*(2**nDim+1), Ns,Ns))
    values_colors = np.reshape(values_colors, (lattice_volume*(2**nDim+1), Nc,Nc))
    return row_index.astype(int), col_index.astype(int), values_spinors, values_colors

In [10]:
def CalculateD_flatten(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli):
    #Delta_x_y, dependent only on the onsite
    diagonal = (m+2)
   
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]
   
    gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
    gauge_links = LatticeToLinear(gauge_links, lattice_volume) * (-0.5)
    gauge_links_shifted_t_T = ConjugateTranspose(LatticeToLinear(gauge_links_shifted_t, lattice_volume) * (-0.5)) #.T correct?
    gauge_links_shifted_x_T = ConjugateTranspose(LatticeToLinear(gauge_links_shifted_x, lattice_volume) * (-0.5))
   
    offdiag_x_plus = np.kron(gauge_links[:,1,:,:],offdiagonal_spinor_x_plus) #Careful with U dimension t,x
    offdiag_x_minus = np.kron(gauge_links_shifted_x_T,offdiagonal_spinor_x_minus)
    offdiag_t_plus = np.kron(gauge_links[:,0,:,:],offdiagonal_spinor_t_plus)
    offdiag_t_minus = np.kron(gauge_links_shifted_t_T,offdiagonal_spinor_t_minus)

    #In each row they are 2^d (next Neighbors) + 1 (onsite)
    row_index = np.arange(0,(2**nDim+1)*lattice_volume+1, 2**nDim+1)


    col_index = np.array([])
    values = np.array([])

    for i in range(lattice_volume):
        #Just moving in time and space to the next neighbors considering periodic boundary
        col_index = np.append(col_index,
                                 np.array([(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt
                                           ,(np.floor(i/4)*4)+((i%Nx)-1)%Nx, i,
                                           (np.floor(i/4)*4)+((i%Nx)+1)%Nx, (i%Nx) +((np.floor((i/Nt))+1)%Nt)*Nt]))
        #Choose the right values depending on direction in space-time, according to the order from above in col_index
       
        values = np.append(values, np.array([offdiag_t_minus[i],
                                         offdiag_x_minus[i], diagonal*np.identity(Ns*Nc),
                                                             offdiag_x_plus[i], offdiag_t_plus[i]]))

       
    values = np.reshape(values, (lattice_volume*(2**nDim+1), Ns*Nc,Ns*Nc))
    return row_index.astype(int), col_index.astype(int), values


In [11]:
def SparseMatrixVectorMultiplication(rows, cols, vals, input_vec):
    out_vec = np.zeros(input_vec.shape,dtype = 'complex_')
    for i in range(input_vec[:,0].size):
        row_elem = rows[i]
        num_elements = rows[i+1] - rows[i]

        output = np.zeros(input_vec[0].shape,dtype = 'complex_')
        for j in range(num_elements):
            output = output + np.matmul(vals[row_elem + j],  input_vec[cols[row_elem + j]])
        out_vec[i] = output
    
    return out_vec

In [12]:
def SparseMatrixVectorMultiplication2(rows, cols, values_spinors, values_colors, input_vec):
    out_vec = np.zeros(input_vec.shape,dtype = 'complex_')
    for i in range(input_vec[:,0,0].size):
        row_elem = rows[i]
        num_elements = rows[i+1] - rows[i]

        output = np.zeros(input_vec[0].shape,dtype = 'complex_')
        for j in range(num_elements):
            output = output + np.matmul(np.matmul(values_colors[row_elem + j],  input_vec[cols[row_elem + j]]),
                                        values_spinors[row_elem + j].T)
        out_vec[i] = output
    
    return out_vec

#### Test random U

In [13]:
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
gauge_links = np.random.randn(lattice_volume, nDim, Nc,Nc)
rows, cols, values = CalculateD_flatten(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new_flatten = SparseMatrixVectorMultiplication(rows, cols, values, psi_flatten)
rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new = SparseMatrixVectorMultiplication2(rows, cols, values_spinors, values_colors, psi)

In [14]:
psi_new_reshaped = np.reshape(psi_new_flatten, (lattice_volume,Nc,Ns))
np.sqrt(np.sum((psi_new-psi_new_reshaped)**2))

(5.605801790696639e-15+0j)

#### Test unitary U

In [15]:
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = unitary_group.rvs(Nc)
        
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
rows, cols, values = CalculateD_flatten(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new_flatten = SparseMatrixVectorMultiplication(rows, cols, values, psi_flatten)
rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new = SparseMatrixVectorMultiplication2(rows, cols, values_spinors, values_colors, psi)
psi_new_reshaped = np.reshape(psi_new_flatten, (lattice_volume,Nc,Ns))
np.sqrt(np.sum((psi_new-psi_new_reshaped)**2))

(4.615988440375771e-15-2.4366245757551863e-17j)

# Apply D directly

In [16]:
def applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    if len(gauge_links) ==lattice_volume:
        gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    if len(psi) ==lattice_volume:
        psi = LinearToLattice(psi, Nt,Nx)

    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]

    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)

    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    psi_shifted_m_t = np.roll(psi, 1, axis = 0)
    psi_shifted_p_t = np.roll(psi, -1, axis = 0)
    psi_shifted_m_x = np.roll(psi, 1, axis = 1)
    psi_shifted_p_x = np.roll(psi, -1, axis = 1)


    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
    time_contribution_m = np.matmul(ConjugateTranspose(gauge_links_shifted_t), psi_shifted_m_t)
    time_contribution_p = np.matmul(gauge_links[:,:,0,:,:], psi_shifted_p_t)
    space_contribution_m = np.matmul(ConjugateTranspose(gauge_links_shifted_x), psi_shifted_m_x)
    space_contribution_p = np.matmul(gauge_links[:,:,1,:,:], psi_shifted_p_x)

    # Estos matmuls son diferentes a los matmul
    time_contribution_m = np.matmul(time_contribution_m, Transpose(offdiagonal_spinor_t_minus))
    time_contribution_p = np.matmul(time_contribution_p, Transpose(offdiagonal_spinor_t_plus))
    space_contribution_m = np.matmul(space_contribution_m, Transpose(offdiagonal_spinor_x_minus))
    space_contribution_p = np.matmul(space_contribution_p, Transpose(offdiagonal_spinor_x_plus))
    

    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* (time_contribution_m + 
                                       time_contribution_p + space_contribution_m + space_contribution_p)
    
    return psi_new

In [17]:
def applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    if len(gauge_links) ==lattice_volume:
        gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    if len(psi) ==lattice_volume:
        psi = LinearToLattice(psi, Nt,Nx)

    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]

    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
   
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]


   

   
   
   
    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    psi_shifted_m_t = np.roll(psi, 1, axis = 0)
    psi_shifted_p_t = np.roll(psi, -1, axis = 0)
    psi_shifted_m_x = np.roll(psi, 1, axis = 1)
    psi_shifted_p_x = np.roll(psi, -1, axis = 1)

   
    offdiag_x_plus = np.kron(gauge_links[:,:,1,:,:],offdiagonal_spinor_x_plus) #Careful with U dimension t,x
    offdiag_x_minus = np.kron(ConjugateTranspose(gauge_links_shifted_x),offdiagonal_spinor_x_minus)
    offdiag_t_plus = np.kron(gauge_links[:,:,0,:,:],offdiagonal_spinor_t_plus)
    offdiag_t_minus = np.kron(ConjugateTranspose(gauge_links_shifted_t),offdiagonal_spinor_t_minus)
   

    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
    time_contribution_m = np.matmul(offdiag_t_minus, psi_shifted_m_t[:,:,:,None]) #Do this without None
    time_contribution_p = np.matmul(offdiag_t_plus, psi_shifted_p_t[:,:,:,None])
    space_contribution_m = np.matmul(offdiag_x_minus, psi_shifted_m_x[:,:,:,None])
    space_contribution_p = np.matmul(offdiag_x_plus, psi_shifted_p_x[:,:,:,None])



    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* np.squeeze((time_contribution_m +  #Squeeze is weird!
                                       time_contribution_p + space_contribution_m + space_contribution_p))
   
    return psi_new

#### Test random U

In [18]:
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
gauge_links = np.random.randn(lattice_volume, nDim, Nc,Nc)
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
psi_new2_flatten = applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi_flatten)

psi_new2_reshaped = np.reshape(psi_new2_flatten, (Nt,Nx,Nc,Ns))
np.sqrt(np.sum((psi_new2 - psi_new2_reshaped)**2))

3.294389233622972e-15

#### Test unitary U

In [19]:
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = unitary_group.rvs(Nc)
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
psi_new2_flatten = applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi_flatten)

psi_new2_reshaped = np.reshape(psi_new2_flatten, (Nt,Nx,Nc,Ns))
np.sqrt(np.sum((psi_new2 - psi_new2_reshaped)**2))

(1.4214727742502255e-15+1.1814583732646566e-16j)